In [1]:
import pandas as pd           
import praw                   
import re                     

import seaborn as sns

## acknowledgements
'''
https://stackoverflow.com/questions/48358837/pulling-reddit-comments-using-python-praw-and-creating-a-dataframe-with-the-resu
https://www.reddit.com/r/redditdev/comments/2e2q2l/praw_downvote_count_always_zero/
https://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91
'''

'\nhttps://stackoverflow.com/questions/48358837/pulling-reddit-comments-using-python-praw-and-creating-a-dataframe-with-the-resu\nhttps://www.reddit.com/r/redditdev/comments/2e2q2l/praw_downvote_count_always_zero/\nhttps://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91\n'

In [4]:
with open("API.env") as file:
    exec(file.read())

reddit = praw.Reddit(
  client_id = client_id,
  client_secret = client_secret,
  user_agent = user_agent
)

Version 7.0.0 of praw is outdated. Version 7.2.0 was released Wednesday February 24, 2021.


In [5]:
'''
Some helper functions for the reddit API.
'''

def extract_num_rewards(awardings_data):
    return sum( x["count"] for x in awardings_data)

def extract_data(submission, comments = False):
    postlist = []

    # extracts top level comments
    if comments:
        submission.comments.replace_more(limit=0)
        for comment in submission.comments: 
            post = {} # put this here
            post['author'] = comment.author
            post['body'] = comment.body
            post['awards'] = comment.all_awardings
            post['score'] = comment.score
            post['parent_id'] = comment.parent_id
            post['id'] = comment.id

            postlist.append(post)

    content = {
    "title" : submission.title,
    "self" : submission.is_self,
    "text" : submission.selftext,
    "comments" : postlist,
    "author" : submission.author,
    "name" : submission.name,
    "upvote_ratio" : submission.upvote_ratio,
    "ups" : submission.score, #this is the same as submission.ups,
    "downs" : None,
    "awarders" : submission.awarders, 
    "awards" : submission.all_awardings,
    "total_awards" : None,
    "url" : submission.url # Only relevent if not a self post
    }
    
    content["total_awards"] = extract_num_rewards(content["awards"])
    content["downs"] = (1 - content["upvote_ratio"]) * content["ups"] / content["upvote_ratio"]
    return content

In [24]:
'''
Sample num_samples random submissions, and get the top num_samples submissions, and put them into dataframes.
'''

def random_sample(num_samples, subreddit):
    sample = []
    for i in range(num_samples):
        submission = reddit.subreddit(subreddit).random() 
        sample.append(extract_data(submission))
    return(pd.DataFrame(sample))
    
def top_sample(num_samples, subreddit):
    top_submissions = []
    for submission in reddit.subreddit(subreddit).top(limit=num_samples):
        top_submissions.append(extract_data(submission))
    print(f"Got {len(top_submissions)} submissions from the top. (This can be less than num_samples.)")
    return(pd.DataFrame(top_submissions))

def rising_sample(num_samples, subreddit):
    # NB: There is also rising_random
    rising = []
    for submission in reddit.subreddit(subreddit).rising(limit=num_samples):
        rising.append(extract_data(submission))
    print(f"Got {len(rising)} submissions from rising. (This can be less than num_samples.)")
    return(pd.DataFrame(rising))

num_samples = 4
subreddit ='wallstreetbets'

random_wsb = random_sample(num_samples, subreddit)
top_wsb = top_sample(num_samples,subreddit)
rising_wsb = rising_sample(num_samples, subreddit)
# other commands here: https://praw.readthedocs.io/en/latest/code_overview/models/subreddit.html#praw.models.Subreddit.rising
# NB: The subreddit stream option seems useful.



Got 4 submissions from the top. (This can be less than num_samples.)
Got 4 submissions from rising. (This can be less than num_samples.)


In [33]:
random_wsb.to_pickle("random_wsb.pkl")
top_wsb.to_pickle("top_wsb.pkl")


In [ ]:
#sns.lmplot(data = random_wsb, x = "total_awards", y = "ups")

In [13]:
# traffic = reddit.subreddit(subreddit).traffic() is not available to us, sadly.

In [18]:
wsb = reddit.subreddit(subreddit).rising(limit = 10)


In [20]:
list(wsb)

[Submission(id='n6i28o'),
 Submission(id='n6hwq5'),
 Submission(id='n6i0cs'),
 Submission(id='n6iyw5'),
 Submission(id='n6irbo'),
 Submission(id='n6imtg'),
 Submission(id='n6isa2'),
 Submission(id='n6ini7'),
 Submission(id='n6j2kf'),
 Submission(id='n6ilkv')]